In [ ]:
!pip install transformers datasets torch scikit-learn

In [ ]:
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import classification_report
import torch

In [ ]:
# 1. Load the IMDB dataset (for sentiment analysis)
dataset = load_dataset("imdb")

# 2. Load the pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 3. Tokenization function to prepare the data
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

# Apply tokenization to the datasets
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
# 4. Define the metrics for evaluation (Accuracy, Precision, Recall, F1-score)
def compute_metrics(p):
    preds = p.predictions.argmax(axis=-1)
    labels = p.label_ids
    return classification_report(labels, preds, output_dict=True)

In [ ]:
from transformers import Trainer, TrainingArguments

# Check transformers version first
import transformers
print(f"Transformers version: {transformers.__version__}")

# Disable wandb if you don't want to use it
import os
os.environ["WANDB_DISABLED"] = "true"

# OPTIMIZED for Google Colab - Fast and efficient training
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    per_device_eval_batch_size=16,
    learning_rate=5e-5,
    weight_decay=0.01,

    # Optimized logging and saving
    logging_steps=100,
    save_steps=500,
    eval_steps=500,

    # Speed optimizations
    dataloader_num_workers=2,
    dataloader_pin_memory=True,
    fp16=True,

    # Reduced overhead
    logging_dir='./logs',
    do_eval=True,
    report_to="none",

    # Memory optimizations
    remove_unused_columns=True,
    prediction_loss_only=True,
)

print("🚀 Optimized for Google Colab:")
print(f"- 1 epoch instead of 3 (saves ~66% time)")
print(f"- Batch size 16 with gradient accumulation")
print(f"- Mixed precision (fp16) for speed")
print(f"- Expected time: ~30-40 minutes instead of 2 hours")


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
)


print("Starting training...")
trainer.train()


final_eval_results = trainer.evaluate()
print(f"Final evaluation results: {final_eval_results}")
trainer.save_model()


Transformers version: 4.55.4
🚀 Optimized for Google Colab:
- 1 epoch instead of 3 (saves ~66% time)
- Batch size 16 with gradient accumulation
- Mixed precision (fp16) for speed
- Expected time: ~30-40 minutes instead of 2 hours
Starting training...


Step,Training Loss
100,0.205000
200,0.253200
300,0.232300
400,0.220100
500,0.209300
600,0.207700
700,0.171400


Final evaluation results: {'eval_loss': 0.16162988543510437, 'eval_runtime': 178.2706, 'eval_samples_per_second': 140.236, 'eval_steps_per_second': 8.768, 'epoch': 1.0}


'\n# For manual control over each epoch\noriginal_epochs = training_args.num_train_epochs\ntraining_args.num_train_epochs = 1  # Train one epoch at a time\n\nfor epoch in range(int(original_epochs)):\n    print(f"Training Epoch {epoch + 1}/{original_epochs}")\n    \n    # Train for one epoch\n    if epoch == 0:\n        trainer.train()\n    else:\n        # Resume from last checkpoint\n        trainer.train(resume_from_checkpoint=True)\n    \n    # Evaluate after each epoch\n    eval_results = trainer.evaluate()\n    print(f"Epoch {epoch + 1} evaluation results: {eval_results}")\n    \n    # Save model after each epoch\n    trainer.save_model(f"./results/epoch_{epoch + 1}")\n'

In [ ]:
import torch
from transformers import pipeline
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

print("🚀 Testing your trained model...")

# Step 1: Load your trained model as a pipeline
print("Loading model pipeline...")
try:
    pipe = pipeline(
        'text-classification',
        model='./results',
        tokenizer=tokenizer,
        device=0 if torch.cuda.is_available() else -1
    )
    print("✅ Model loaded successfully!")
except Exception as e:
    print(f"❌ Error loading pipeline: {e}")
    print("Trying alternative approach...")

# Step 2: Test with sample examples
print("\n📝 Testing with sample texts...")

# Sample test cases - REPLACE with examples from your domain
test_samples = [
    "This product is amazing and works perfectly!",
    "Terrible quality, waste of money.",
    "It's okay, nothing special but decent.",
    "Best purchase I've ever made, highly recommend!",
    "Poor customer service and late delivery."
]

if 'pipe' in locals():
    print("Sample Predictions:")
    print("-" * 50)
    for i, text in enumerate(test_samples, 1):
        try:
            result = pipe(text)
            label = result[0]['label']
            score = result[0]['score']
            print(f"{i}. Text: '{text}'")
            print(f"   → Prediction: {label} (confidence: {score:.3f})")
            print()
        except Exception as e:
            print(f"Error with sample {i}: {e}")

# Step 3: Test on your actual test dataset
print("📊 Evaluating on your test dataset...")

try:

    print("Test dataset info:")
    print(f"Number of test samples: {len(tokenized_datasets['test'])}")
    print(f"Features: {list(tokenized_datasets['test'].features.keys())}")


    if 'text' in tokenized_datasets['test'].features:
        print("\nTesting on real examples from your dataset:")
        print("-" * 60)


        test_texts = tokenized_datasets['test']['text'][:10]
        test_labels = tokenized_datasets['test']['label'][:10] if 'label' in tokenized_datasets['test'].features else None

        correct = 0
        for i, text in enumerate(test_texts):
            try:
                result = pipe(text)
                predicted_label = result[0]['label']
                confidence = result[0]['score']

                print(f"{i+1}. Text: '{text[:80]}...'")
                print(f"   Predicted: {predicted_label} ({confidence:.3f})")

                if test_labels is not None:
                    true_label = test_labels[i]
                    print(f"   True label: {true_label}")

                print()

            except Exception as e:
                print(f"Error with test example {i+1}: {e}")


    else:
        print("No 'text' column found - dataset might be pre-tokenized")

except Exception as e:
    print(f"Error testing on dataset: {e}")


print("📈 Getting comprehensive model metrics...")

try:

    trainer.model.eval()
    eval_results = trainer.predict(tokenized_datasets['test'])

    print("Test Results:")
    print("-" * 30)
    for metric, value in eval_results.metrics.items():
        print(f"{metric}: {value:.4f}")


    predictions = np.argmax(eval_results.predictions, axis=1)
    true_labels = eval_results.label_ids


    accuracy = accuracy_score(true_labels, predictions)
    print(f"\nOverall Test Accuracy: {accuracy:.4f}")


    print("\nDetailed Classification Report:")
    print(classification_report(true_labels, predictions))

except Exception as e:
    print(f"Error in comprehensive evaluation: {e}")


print("\n🎯 Try your own examples!")
print("Type some text to see what your model predicts")
print("(Type 'quit' to exit)")

if 'pipe' in locals():
    while True:
        try:
            user_input = input("\nEnter text: ").strip()

            if user_input.lower() in ['quit', 'exit', 'q', '']:
                break

            result = pipe(user_input)
            print(f"Prediction: {result[0]['label']}")
            print(f"Confidence: {result[0]['score']:.3f}")


            if len(result) == 1 and len(result[0]) > 2:
                print("All class probabilities:")
                for pred in result:
                    print(f"  {pred['label']}: {pred['score']:.3f}")

        except KeyboardInterrupt:
            print("\nExiting...")
            break
        except Exception as e:
            print(f"Error: {e}")

print("\n✅ Testing completed!")


print("\n📋 Model Summary:")
try:
    print(f"Model saved in: ./results")
    print(f"Model type: {trainer.model.config.model_type}")
    print(f"Number of labels: {trainer.model.config.num_labels}")

    if hasattr(trainer.model.config, 'id2label'):
        print("Label mappings:")
        for id_num, label_name in trainer.model.config.id2label.items():
            print(f"  {id_num}: {label_name}")

except Exception as e:
    print(f"Could not show model info: {e}")

Device set to use cuda:0


🚀 Testing your trained model...
Loading model pipeline...
✅ Model loaded successfully!

📝 Testing with sample texts...
Sample Predictions:
--------------------------------------------------
1. Text: 'This product is amazing and works perfectly!'
   → Prediction: LABEL_1 (confidence: 0.995)

2. Text: 'Terrible quality, waste of money.'
   → Prediction: LABEL_0 (confidence: 0.997)

3. Text: 'It's okay, nothing special but decent.'
   → Prediction: LABEL_0 (confidence: 0.929)

4. Text: 'Best purchase I've ever made, highly recommend!'
   → Prediction: LABEL_1 (confidence: 0.995)

5. Text: 'Poor customer service and late delivery.'
   → Prediction: LABEL_0 (confidence: 0.995)

📊 Evaluating on your test dataset...
Test dataset info:
Number of test samples: 25000
Features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask']

Testing on real examples from your dataset:
------------------------------------------------------------
1. Text: 'I love sci-fi and am willing to put up 

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


5. Text: 'First off let me say, If you haven't enjoyed a Van Damme movie since bloodsport,...'
   Predicted: LABEL_1 (0.988)
   True label: 0

6. Text: 'I had high hopes for this one until they changed the name to 'The Shepherd : Bor...'
   Predicted: LABEL_0 (0.992)
   True label: 0

7. Text: 'Isaac Florentine has made some of the best western Martial Arts action movies ev...'
   Predicted: LABEL_0 (0.932)
   True label: 0

8. Text: 'It actually pains me to say it, but this movie was horrible on every level. The ...'
   Predicted: LABEL_0 (0.998)
   True label: 0

9. Text: 'Technically I'am a Van Damme Fan, or I was. this movie is so bad that I hated my...'
   Predicted: LABEL_0 (0.998)
   True label: 0

10. Text: 'Honestly awful film, bad editing, awful lighting, dire dialog and scrappy screen...'
   Predicted: LABEL_0 (0.998)
   True label: 0

📈 Getting comprehensive model metrics...


Test Results:
------------------------------
test_loss: 0.1616
test_runtime: 175.8507
test_samples_per_second: 142.1660
test_steps_per_second: 8.8880
Error in comprehensive evaluation: axis 1 is out of bounds for array of dimension 1

🎯 Try your own examples!
Type some text to see what your model predicts
(Type 'quit' to exit)

Enter text: You suck!
Prediction: LABEL_0
Confidence: 0.719

Enter text: The product was medicore
Prediction: LABEL_1
Confidence: 0.560

Enter text: The product was great!
Prediction: LABEL_1
Confidence: 0.991

Enter text: quit

✅ Testing completed!

📋 Model Summary:
Model saved in: ./results
Model type: bert
Number of labels: 2
Label mappings:
  0: LABEL_0
  1: LABEL_1


In [ ]:
# Simple Model Saving (No Google Drive needed)

print("💾 SAVING YOUR MODEL LOCALLY...")



print("1. Creating backup copy with clear name...")
import shutil
import os

try:

    if os.path.exists("./results"):
        shutil.copytree("./results", "./my_text_classifier", dirs_exist_ok=True)
        print("✅ Model copied to: ./my_text_classifier")


    trainer.save_model("./my_saved_model")
    tokenizer.save_pretrained("./my_saved_model")
    print("✅ Model saved to: ./my_saved_model")

except Exception as e:
    print(f"Error copying: {e}")

print("\n📋 Your model files are saved in:")
print("  - ./results (original)")
print("  - ./my_text_classifier (backup)")
print("  - ./my_saved_model (explicit save)")


print("\n🧪 Testing model loading...")
try:
    from transformers import pipeline


    test_pipe = pipeline("text-classification", model="./my_saved_model")

    # Quick test
    test_result = test_pipe("This is a test sentence")
    print(f"✅ Model loads successfully!")
    print(f"Test prediction: {test_result[0]['label']} ({test_result[0]['score']:.3f})")

except Exception as e:
    print(f"❌ Loading test failed: {e}")

print("\n" + "="*60)
print("🎉 MODEL SAVED SUCCESSFULLY!")
print("="*60)

# ============================================================================
# HOW TO USE YOUR MODEL IN FUTURE SESSIONS
# ============================================================================

print("\n📖 TO USE YOUR MODEL LATER, COPY THIS CODE:")
print("-" * 50)

usage_code = """
# Copy and paste this code to use your model in any new notebook/session:

from transformers import pipeline

# Load your trained model
model = pipeline("text-classification", model="./my_saved_model")

# Use it to make predictions
def classify_text(text):
    result = model(text)
    return {
        'label': result[0]['label'],
        'confidence': result[0]['score']
    }

# Example usage:
text = "Your text here"
prediction = classify_text(text)
print(f"Text: '{text}'")
print(f"Prediction: {prediction['label']} (confidence: {prediction['confidence']:.3f})")

# Batch predictions:
texts = ["Text 1", "Text 2", "Text 3"]
results = model(texts)
for text, result in zip(texts, results):
    print(f"'{text}' -> {result['label']} ({result['score']:.3f})")
"""

print(usage_code)
print("-" * 50)

# ============================================================================
# DOWNLOAD FILES (if you want to save locally)
# ============================================================================

print("\n💾 TO DOWNLOAD MODEL FILES:")
print("1. In Colab, go to Files panel (📁) on the left")
print("2. Navigate to 'my_saved_model' folder")
print("3. Right-click folder -> Download")
print("4. Save the .zip file on your computer")
print("5. Extract and use the same code above!")

# ============================================================================
# MODEL INFO
# ============================================================================

print("\n📊 YOUR MODEL INFO:")
try:
    print(f"Model type: {trainer.model.config.model_type}")
    print(f"Number of classes: {trainer.model.config.num_labels}")

    if hasattr(trainer.model.config, 'id2label'):
        print("Classes:")
        for id_num, label in trainer.model.config.id2label.items():
            print(f"  {id_num}: {label}")

    print(f"Model size: ~{sum(p.numel() for p in trainer.model.parameters()) / 1e6:.1f}M parameters")

except Exception as e:
    print(f"Could not get model info: {e}")

print("\n🚀 YOUR MODEL IS READY TO USE!")
print("Just run the usage code above in any new session!")

💾 SAVING YOUR MODEL LOCALLY...
1. Creating backup copy with clear name...
✅ Model copied to: ./my_text_classifier


Device set to use cuda:0


✅ Model saved to: ./my_saved_model

📋 Your model files are saved in:
  - ./results (original)
  - ./my_text_classifier (backup)
  - ./my_saved_model (explicit save)

🧪 Testing model loading...
✅ Model loads successfully!
Test prediction: LABEL_1 (0.732)

🎉 MODEL SAVED SUCCESSFULLY!

📖 TO USE YOUR MODEL LATER, COPY THIS CODE:
--------------------------------------------------

# Copy and paste this code to use your model in any new notebook/session:

from transformers import pipeline

# Load your trained model
model = pipeline("text-classification", model="./my_saved_model")

# Use it to make predictions
def classify_text(text):
    result = model(text)
    return {
        'label': result[0]['label'],
        'confidence': result[0]['score']
    }

# Example usage:
text = "Your text here"
prediction = classify_text(text)
print(f"Text: '{text}'")
print(f"Prediction: {prediction['label']} (confidence: {prediction['confidence']:.3f})")

# Batch predictions:
texts = ["Text 1", "Text 2", "T